In [1]:
import duckdb
import pyarrow as pa

In [22]:
# create dummy tables
table1 = pa.Table.from_pydict({
    'token_1': [1, 2, 3, 4],
    'value':   [1, 2, 3, 4],
})
table2 = pa.Table.from_pydict({
    'token_1': [1, 2, 3, 4, 1, 2, 3, 4],
    'token_2': [1, 1, 1, 1, 3, 3, 3, 3],
    'value':   [0, 0, 0, 0, 0, 0, 0, 0] # set a default value smaller than any possible value so I can take the maximum
})
table3 = pa.Table.from_pydict({
    'token_1': [1, 2, 3, 4, 1, 2, 3, 4],
    'token_2': [1, 1, 1, 1, 3, 3, 3, 3],
    'token_3': [1, 3, 1, 3, 1, 3, 1, 3],
    'value':   [0, 0, 0, 0, 0, 0, 0, 0] # set a default value smaller than any possible value so I can take the maximum
})

In [23]:
create_table_queries = [
    'CREATE OR REPLACE TABLE t1 AS SELECT * FROM table1',
    'CREATE OR REPLACE TABLE t2 AS SELECT * FROM table2',
    'CREATE OR REPLACE TABLE t3 AS SELECT * FROM table3',
]
for query in create_table_queries:
    duckdb.execute(query)

In [24]:
def get_table(table_name: str):
    return duckdb.sql(f'SELECT * FROM {table_name}')

In [25]:
get_table('t1')

┌─────────┬───────┐
│ token_1 │ value │
│  int64  │ int64 │
├─────────┼───────┤
│       1 │     1 │
│       2 │     2 │
│       3 │     3 │
│       4 │     4 │
└─────────┴───────┘

In [26]:
get_table('t2')

┌─────────┬─────────┬───────┐
│ token_1 │ token_2 │ value │
│  int64  │  int64  │ int64 │
├─────────┼─────────┼───────┤
│       1 │       1 │     0 │
│       2 │       1 │     0 │
│       3 │       1 │     0 │
│       4 │       1 │     0 │
│       1 │       3 │     0 │
│       2 │       3 │     0 │
│       3 │       3 │     0 │
│       4 │       3 │     0 │
└─────────┴─────────┴───────┘

In [27]:
get_table('t3')

┌─────────┬─────────┬─────────┬───────┐
│ token_1 │ token_2 │ token_3 │ value │
│  int64  │  int64  │  int64  │ int64 │
├─────────┼─────────┼─────────┼───────┤
│       1 │       1 │       1 │     0 │
│       2 │       1 │       3 │     0 │
│       3 │       1 │       1 │     0 │
│       4 │       1 │       3 │     0 │
│       1 │       3 │       1 │     0 │
│       2 │       3 │       3 │     0 │
│       3 │       3 │       1 │     0 │
│       4 │       3 │       3 │     0 │
└─────────┴─────────┴─────────┴───────┘

In [19]:
# dynamic programming with SQL
# ngrams of size 2

In [47]:
# get the row of values from t1 that matches the first token
select_value1_query = 'SELECT t2.token_1 AS token_1, t1.value AS value1 FROM t2 INNER JOIN t1 ON t2.token_1 = t1.token_1'
duckdb.sql(select_value1_query)

┌─────────┬────────┐
│ token_1 │ value1 │
│  int64  │ int64  │
├─────────┼────────┤
│       1 │      1 │
│       2 │      2 │
│       3 │      3 │
│       4 │      4 │
│       1 │      1 │
│       2 │      2 │
│       3 │      3 │
│       4 │      4 │
└─────────┴────────┘

In [48]:
# get the row of values from t1 that matches the second token
select_value2_query = 'SELECT t2.token_2 AS token_2, t1.value AS value2 FROM t2 INNER JOIN t1 ON t2.token_2 = t1.token_1'
duckdb.sql(select_value2_query)

┌─────────┬────────┐
│ token_2 │ value2 │
│  int64  │ int64  │
├─────────┼────────┤
│       1 │      1 │
│       1 │      1 │
│       1 │      1 │
│       1 │      1 │
│       3 │      3 │
│       3 │      3 │
│       3 │      3 │
│       3 │      3 │
└─────────┴────────┘

In [49]:
# get the sum of the values
select_sum_value = f"""
SELECT token_1, token_2, value1 + value2 AS sum_value 
FROM ({select_value1_query}) POSITIONAL JOIN ({select_value2_query})
"""
duckdb.sql(select_sum_value)

┌─────────┬─────────┬───────────┐
│ token_1 │ token_2 │ sum_value │
│  int64  │  int64  │   int64   │
├─────────┼─────────┼───────────┤
│       1 │       1 │         2 │
│       2 │       1 │         3 │
│       3 │       1 │         4 │
│       4 │       1 │         5 │
│       1 │       3 │         4 │
│       2 │       3 │         5 │
│       3 │       3 │         6 │
│       4 │       3 │         7 │
└─────────┴─────────┴───────────┘

In [51]:
# update the table with the maximum sum value
max_update_query = f"""
UPDATE t2
SET value = greatest(t2.value, sum_value) 
FROM ({select_sum_value}) AS x
WHERE t2.token_1 = x.token_1 AND t2.token_2 = x.token_2
"""
duckdb.execute(max_update_query)

In [52]:
get_table('t2')

┌─────────┬─────────┬───────┐
│ token_1 │ token_2 │ value │
│  int64  │  int64  │ int64 │
├─────────┼─────────┼───────┤
│       1 │       1 │     2 │
│       2 │       1 │     3 │
│       3 │       1 │     4 │
│       4 │       1 │     5 │
│       1 │       3 │     4 │
│       2 │       3 │     5 │
│       3 │       3 │     6 │
│       4 │       3 │     7 │
└─────────┴─────────┴───────┘

In [68]:
q = f"""
UPDATE t3 
    SET value = greatest(x.value, t2.value + t1.value)
FROM t3 x
    INNER JOIN t2 ON (x.token_1 = t2.token_1 AND x.token_2 = t2.token_2)
    INNER JOIN t1 ON (x.token_3 = t1.token_1)
WHERE (t3.token_1 = x.token_1 AND t3.token_2 = x.token_2 AND t3.token_3 = x.token_3) 
"""
duckdb.execute(q)
get_table('t3')

┌─────────┬─────────┬─────────┬───────┐
│ token_1 │ token_2 │ token_3 │ value │
│  int64  │  int64  │  int64  │ int64 │
├─────────┼─────────┼─────────┼───────┤
│       1 │       1 │       1 │     3 │
│       2 │       1 │       3 │     6 │
│       3 │       1 │       1 │     5 │
│       4 │       1 │       3 │     8 │
│       1 │       3 │       1 │     5 │
│       2 │       3 │       3 │     8 │
│       3 │       3 │       1 │     7 │
│       4 │       3 │       3 │    10 │
└─────────┴─────────┴─────────┴───────┘

In [76]:
# ngram of size 3, there are two steps. (token_1, token_2), token_3 | token_1, (token_2, token_3)
def get_update_max_query(ngram_size: int, index_to_slice: int) -> str:
    if index_to_slice > ngram_size - 1 or index_to_slice < 1:
        raise ValueError
    
    tokens = [f'token_{i}' for i in range(1, ngram_size + 1)]
    token_slice1 = tokens[:index_to_slice]
    token_slice2 = tokens[index_to_slice:]
    offseted_token_slice2 = tokens[:len(token_slice2)]
      
    table_to_update = f't{ngram_size}'
    table_1 = f't{len(token_slice1)}'
    table_2 = f't{len(token_slice2)}'
    
    q = f"""
    UPDATE {table_to_update}
        SET value = greatest(x.value, {table_1}.value + {table_2}.value)
    FROM t3 x
        INNER JOIN {table_1} ON 
            ({' AND '.join(f'x.{token} = {table_1}.{token}' for token in token_slice1)})
        INNER JOIN {table_2} ON 
            ({' AND '.join(f'x.{token} = {table_2}.{token_}' for token, token_ in zip(token_slice2, offseted_token_slice2))})
    WHERE ({' AND '.join(f'{table_to_update}.{token} = x.{token}' for token in tokens)})
    """
    return q

In [77]:
print(get_update_max_query(3, 1))


    UPDATE t3
        SET value = greatest(x.value, t1.value + t2.value)
    FROM t3 x
        INNER JOIN t1 ON 
            (x.token_1 = t1.token_1)
        INNER JOIN t2 ON 
            (x.token_2 = t2.token_1 AND x.token_3 = t2.token_2)
    WHERE (t3.token_1 = x.token_1 AND t3.token_2 = x.token_2 AND t3.token_3 = x.token_3)
    


In [78]:
# initialize values to 0
reset_query = 'UPDATE t3 SET value = 0'
duckdb.execute(reset_query)
get_table('t3')

┌─────────┬─────────┬─────────┬───────┐
│ token_1 │ token_2 │ token_3 │ value │
│  int64  │  int64  │  int64  │ int64 │
├─────────┼─────────┼─────────┼───────┤
│       1 │       1 │       1 │     0 │
│       2 │       1 │       3 │     0 │
│       3 │       1 │       1 │     0 │
│       4 │       1 │       3 │     0 │
│       1 │       3 │       1 │     0 │
│       2 │       3 │       3 │     0 │
│       3 │       3 │       1 │     0 │
│       4 │       3 │       3 │     0 │
└─────────┴─────────┴─────────┴───────┘

In [79]:
update_query1 = get_update_max_query(3, 1)
duckdb.execute(update_query1)
get_table('t3')

┌─────────┬─────────┬─────────┬───────┐
│ token_1 │ token_2 │ token_3 │ value │
│  int64  │  int64  │  int64  │ int64 │
├─────────┼─────────┼─────────┼───────┤
│       1 │       1 │       1 │     3 │
│       2 │       1 │       3 │     6 │
│       3 │       1 │       1 │     5 │
│       4 │       1 │       3 │     8 │
│       1 │       3 │       1 │     5 │
│       2 │       3 │       3 │     8 │
│       3 │       3 │       1 │     7 │
│       4 │       3 │       3 │    10 │
└─────────┴─────────┴─────────┴───────┘

In [80]:
duckdb.execute(reset_query)
get_table('t3')

┌─────────┬─────────┬─────────┬───────┐
│ token_1 │ token_2 │ token_3 │ value │
│  int64  │  int64  │  int64  │ int64 │
├─────────┼─────────┼─────────┼───────┤
│       1 │       1 │       1 │     0 │
│       2 │       1 │       3 │     0 │
│       3 │       1 │       1 │     0 │
│       4 │       1 │       3 │     0 │
│       1 │       3 │       1 │     0 │
│       2 │       3 │       3 │     0 │
│       3 │       3 │       1 │     0 │
│       4 │       3 │       3 │     0 │
└─────────┴─────────┴─────────┴───────┘

In [81]:
update_query1 = get_update_max_query(3, 2)
duckdb.execute(update_query1)
get_table('t3')

┌─────────┬─────────┬─────────┬───────┐
│ token_1 │ token_2 │ token_3 │ value │
│  int64  │  int64  │  int64  │ int64 │
├─────────┼─────────┼─────────┼───────┤
│       1 │       1 │       1 │     3 │
│       2 │       1 │       3 │     6 │
│       3 │       1 │       1 │     5 │
│       4 │       1 │       3 │     8 │
│       1 │       3 │       1 │     5 │
│       2 │       3 │       3 │     8 │
│       3 │       3 │       1 │     7 │
│       4 │       3 │       3 │    10 │
└─────────┴─────────┴─────────┴───────┘

In [82]:
duckdb.execute(reset_query)
get_table('t3')

┌─────────┬─────────┬─────────┬───────┐
│ token_1 │ token_2 │ token_3 │ value │
│  int64  │  int64  │  int64  │ int64 │
├─────────┼─────────┼─────────┼───────┤
│       1 │       1 │       1 │     0 │
│       2 │       1 │       3 │     0 │
│       3 │       1 │       1 │     0 │
│       4 │       1 │       3 │     0 │
│       1 │       3 │       1 │     0 │
│       2 │       3 │       3 │     0 │
│       3 │       3 │       1 │     0 │
│       4 │       3 │       3 │     0 │
└─────────┴─────────┴─────────┴───────┘

In [83]:
# now, the dynamic algorithm is to go from 1 to ngram_size - 1 and compute the max of it all
ngram_size = 3
for i in range(1, ngram_size):
    update_max_query = get_update_max_query(ngram_size, i)
    duckdb.execute(update_max_query)
get_table('t3')

┌─────────┬─────────┬─────────┬───────┐
│ token_1 │ token_2 │ token_3 │ value │
│  int64  │  int64  │  int64  │ int64 │
├─────────┼─────────┼─────────┼───────┤
│       1 │       1 │       1 │     3 │
│       2 │       1 │       3 │     6 │
│       3 │       1 │       1 │     5 │
│       4 │       1 │       3 │     8 │
│       1 │       3 │       1 │     5 │
│       2 │       3 │       3 │     8 │
│       3 │       3 │       1 │     7 │
│       4 │       3 │       3 │    10 │
└─────────┴─────────┴─────────┴───────┘